# Example for open_FRED weather data download

This example shows you how to download open_FRED weather data from the [OpenEnergy DataBase](https://openenergy-platform.org/dataedit/) and store it locally. Furthermore, it shows how to convert the weather data to the format needed by the pvlib and windpowerlib.

In [1]:
# imports
from shapely.geometry import Point, Polygon

from feedinlib.open_FRED import Weather
from feedinlib.open_FRED import defaultdb

## Download data for single coordinate

In [2]:
location = Point(13.5, 52.4)

Besides a location you have to specify a time period for which you would like to download the data as well as the weather variables you need. The feedinlib provides predefined sets of variables that are needed to use the pvlib and windpowerlib. These can be applied by setting the `variables` parameter to "pvlib" or "windpowerlib", as shown below.

In [3]:
# download data for January 2015 (end date will not be included in the 
# time period for which data is downloaded)
start_date, end_date = '2015-01-01', '2015-02-01'
variables = "windpowerlib"

As the open_FRED weather dataset provides some variables at different heights, such as wind speed and air pressure, it is possible to define which heights you want to retrieve the data for.

In [4]:
heights = [80, 100]

Now we can retrieve the data:

In [5]:
open_FRED_weather_windpowerlib = Weather(
    start=start_date, stop=end_date, locations=[location],
    heights=heights,
    variables=variables,
    **defaultdb())

## Download data for a region

In [6]:
lat_point_list = [51.936377, 51.936377, 51.744302, 51.744302, 51.936377]
lon_point_list = [12.621739, 13.005414, 13.005414, 12.621739, 12.621739]
region = Polygon(zip(lon_point_list, lat_point_list))

In this example we will retrieve weather data needed for pvlib calculations. We can do this by setting `variables` to "pvlib". In this case specifying the heights for which to retrieve the data is not necessary as irradiance data is only available at the surface and 10m wind speed is used per default.

**<font color=red>The following code may take a while to execute!</font>**

In [7]:
open_FRED_weather_pvlib = Weather(
    start='2015-06-01', stop='2015-06-05', 
    locations=[], regions=[region],
    variables="pvlib",
    **defaultdb())

## Convert data into pvlib and windpowerlib format

In order to use the weather data for your feed-in calculations using the pvlib and windpowerlib it has to be converted into the required format. This can easily be done as follows. 

In [8]:
# convert to windpowerlib dataframe
windpowerlib_df = open_FRED_weather_windpowerlib.df(location=location, lib="windpowerlib")

In [9]:
# save dataframe as csv
windpowerlib_df.to_csv('windpowerlib_df.csv')

In [11]:
# select point inside region
point = Point(12.7, 51.9)
# convert to pvlib dataframe
pvlib_df = open_FRED_weather_pvlib.df(location=point, lib="pvlib")